# resaurantRating

## ETL Group Project

    Germaine Johnson, Jeremy Jones, Reza Abasaltian
    October 27, 2020

### Google Places API - Text Search

In [1]:
# Import dependencies
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime, timedelta

# Google developer API key
from api_key import gkey

# Set today as current date and time
t = datetime.now()

# Print todays date formatted as mm/dd/yy
date = t.strftime('%m/%d/%y')

# format time to round to the nearest hour in hundreds
time = (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
                        + timedelta(hours=t.minute//30))
hour = time.strftime('%H'+'00')

print(f'Today is {date} @ {hour} hour.')

Today is 10/23/20 @ 1800 hour.


In [2]:
# assign zip code for the base of our search
target_zip = "77056"

# distance, IN METERS, within which the place results must live from assigned zip code
target_radius = 11111

# type of establishment to filter place results
target_type = "restaurant"

In [3]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

In [4]:
# set up a dictionary to store all query parameters - for initial page and first 20 listings
params = {
    "query": target_zip,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# print the response status code
print(f'first response: {response.status_code}')

first response: 200


In [5]:
# convert the response to json
response_json = response.json()

In [6]:
print(f'Total {target_type} listings retrieved on first response: {len(response_json["results"])}')

Total restaurant listings retrieved on first response: 0


In [7]:
# set up a dictionary to store all query parameters - next page and next 20 listings, second response
params = {
    "query": target_zip,
    "radius": target_radius,
    "type": target_type,
    "key": gkey,
    "pagetoken": response_json['next_page_token']
}

# run a request using our params dictionary
response2 = requests.get(base_url, params=params)

# print the response status code
print(f'second response: {response2.status_code}')

# convert the response to json
response_json2 = response2.json()

KeyError: 'next_page_token'

In [8]:
# set up a dictionary to store all query parameters - next page and next 20 listings, third response
params = {
    "query": target_zip,
    "radius": target_radius,
    "type": target_type,
    "key": gkey,
    "pagetoken": response_json2['next_page_token']
}

# run a request using our params dictionary
response3 = requests.get(base_url, params=params)

# print the response status code
print(f'third response: {response3.status_code}')

# convert the response to json
response_json3 = response3.json()

NameError: name 'response_json2' is not defined

In [9]:
def getPlaces(response_json, i, date, hour):
    places = []
    for result in response_json['results']:
        place = {}
        try:
            place['id'] = i
            place['name'] = result['name']
            address = result['formatted_address']
            s = address.split(', ')
            s2 = s[2].split(' ')   
            place['street'] = s[0]
            place['city'] = s[1]
            place['state'] = s2[0]
            place['zip code'] = s2[1]
            place['avg rating'] = result['rating']
            place['total ratings'] = result['user_ratings_total']
            place['price level'] = result['price_level']
            place['date'] = date
            place['hour'] = hour
            places.append(place)
            i+=1
        
        except (KeyError, IndexError) as e:
            if str(e) == "'price_level'":
                place['price level'] = "NA"                  
                places.append(place)
                print(f'Missing field/result... set NA. {str(e)}, listing {i}')
                i+=1
            else:
                print(f'Missing field/result... skipping. {str(e)}')        
    return places

In [10]:
# call get place function for each response
df_places1 = pd.DataFrame(getPlaces(response_json,0,date,hour))
df_places2 = pd.DataFrame(getPlaces(response_json2,len(df_places1),date,hour))
df_places3 = pd.DataFrame(getPlaces(response_json3,(len(df_places1)+len(df_places2)),date,hour))

NameError: name 'response_json2' is not defined

In [11]:
# Concatenate all 3 API responses
df_places = pd.concat([df_places1, df_places2, df_places3], axis=0)
df_places = df_places.reset_index(drop=True)
df_places.head(10)

NameError: name 'df_places2' is not defined

In [12]:
# sorted dataframe by total ratings
df_ratings = df_places.sort_values(by='total ratings', ascending=False)
df_ratings = df_ratings.reset_index(drop=True)
df_ratings.head(10)

NameError: name 'df_places' is not defined

In [13]:
#create specific columns from df
ratings_columns = ["id", "name", "street", "city", "state", "zip code", "avg ratings", "total ratings", "price level", "date", "hour"]
ratings_transformed = df_ratings[ratings_columns].copy()

#Rename the column headers
ratings_transformed = ratings_transformed.rename(columns={"id": "ID",
                                                         "name": "Restaurant Name",
                                                         "street": "Street Address",
                                                         "city": "City", 
                                                         "state": "State", 
                                                         "zip code": "Zip Code", 
                                                         "avg ratings": "Avg Ratings", 
                                                         "total ratings": "Total Ratings", 
                                                         "price level": "Price Level",
                                                         "date": "Date",
                                                         "hour": "Hour"})

# Clean the data by dropping duplicates and setting the index
#ratings_transformed.drop_duplicates("id", inplace=True)
ratings_transformed.set_index("id", inplace=True)

ratings_transformed.head()

NameError: name 'df_ratings' is not defined